In [ ]:
import pandas as pd
import seaborn as sns

df = pd.read_csv('results.csv')
gs = pd.read_csv('eng-premier-league.csv')

#clean gs
#split 'game' field into home_team and away
teams = gs['game'].str.split('vs.', expand=True)
gs['home_team'] = teams[0].str.strip().str.lower()
away_with_score = teams[1].str.strip().str.lower()

#take out scoreline the from away
away_split = away_with_score.str.split()
sliced_away = away_split.str.slice(0, -1)
gs['away_team'] = sliced_away.str.join(' ')

#take out 'fc' from home and away for better merge
home_split = gs['home_team'].str.split()
away_split = gs['away_team'].str.split()
scoring_split = gs['scoring_team'].str.split()
take_home_fc = home_split.str[-1] == 'fc'
take_away_fc = away_split.str[-1] == 'fc'
take_scoring_fc = scoring_split.str[-1] == 'FC'
home_split[take_home_fc] = home_split[take_home_fc].str.slice(0, -1)
away_split[take_away_fc] = away_split[take_away_fc].str.slice(0, -1)
scoring_split[take_scoring_fc] = scoring_split[take_scoring_fc].str.slice(0, -1)
gs['home_team'] = home_split.str.join(' ')
gs['away_team'] = away_split.str.join(' ')
gs['scoring_team'] = scoring_split.str.join(' ')

#dictionary to match found mismatches
team_name_fixes = {'brighton & hove albion': 'brighton hove albion', 'wimbledon': 'afc wimbledon'}
gs['home_team'] = gs['home_team'].replace(team_name_fixes)
gs['away_team'] = gs['away_team'].replace(team_name_fixes)

#get unique game id
gs['game'] = gs['home_team'] + ' vs. ' + gs['away_team']
gs['date'] = gs['date'].str.strip()
gs['gameid'] = gs['game'] + gs['date']

#clean df
#match home and away
df['home'] = df['home'].str.strip().str.lower()
df['away'] = df['away'].str.strip().str.lower()

#take out fc for better merge
home_split_df = df['home'].str.split()
away_split_df = df['away'].str.split()
take_home_fc_df = home_split_df.str[-1] == 'fc'
take_away_fc_df = away_split_df.str[-1] == 'fc'
home_split_df[take_home_fc_df] = home_split_df[take_home_fc_df].str.slice(0, -1)
away_split_df[take_away_fc_df] = away_split_df[take_away_fc_df].str.slice(0, -1)
df['home'] = home_split_df.str.join(' ')
df['away'] = away_split_df.str.join(' ')

#get unique game id
df['game'] = df['home'] + ' vs. ' + df['away']
df['date'] = df['date'].str.strip()
df['gameid'] = df['game'] + df['date']

#filter datasets to 1990-2023 (AI bot helped me with .dt.year.between)
gs['date_dt'] = pd.to_datetime(gs['date'], errors='coerce')
df['date_dt'] = pd.to_datetime(df['date'], errors='coerce')
gs = gs[gs['date_dt'].dt.year.between(1990, 2023)]
df = df[df['date_dt'].dt.year.between(1990, 2023)]

gs = gs.drop(columns=['GH', 'GA'], errors='ignore') # remove old rows
gs = gs.merge(df[['gameid', 'gh', 'ga']], on='gameid', how='left') #merge new rows

def determine_winner(row): #function to extract match winner
    if row['gh'] > row['ga']:
        return row['home_team']
    elif row['gh'] < row['ga']:
        return row['away_team']
    else:
        return 'Draw'

gs['winner'] = gs.apply(determine_winner, axis=1) #create winner field

In [ ]:
import matplotlib.pyplot as plt
from ipywidgets import interact_manual
import ipywidgets as widgets

def visualize(club_df, club, season):
    plt.figure(figsize=(20, 18)) #make dashboard for multiple viz

    plt.subplot(3, 2, 1) #make first viz
    club_goals = club_df[club_df['scoring_team'] == club] #filter for clubs goals
    top_scorers = club_goals['scoring_player'].value_counts().index[:10] #get top scorers
    if len(top_scorers) == 0:
        plt.title(f"No scorer data found for {club.title()} in ({season}) Season")
    else:     
        sns.countplot(data=club_goals[club_goals['scoring_player'].isin(top_scorers)], y='scoring_player', order=top_scorers) #plot top scorers in order
        plt.title(f"Top Scorers for {club.title()} in ({season}) Season")
    
    plt.subplot(3, 2, 2) #make second viz
    results = [] #make lists for results
    for _, match in club_df.iterrows():
        if match['winner'] == club:
            results.append('Win') #make win field
        elif match['winner'] == 'Draw':
            results.append('Draw') #make draw field
        else:
            results.append('Loss') #make loss field
    results_df = pd.DataFrame({'Result': results}) #make a results df
    colors = {'Win': 'green', 'Draw': 'gray', 'Loss': 'red'} #make dict for viz colors
    sns.countplot(data=results_df, x='Result', hue='Result', palette=colors, legend=False) #plot result distribution and make color based on result
    plt.title(f'Match Results Distribution for {club.title()} in {season} Season')

    plt.subplot(3, 2, 3) #make third viz
    home_goals = club_df[club_df['home_team'] == club]['gh'] #make home goals variable
    away_goals = club_df[club_df['away_team'] == club]['ga'] #make away goals variable
    goals_data = pd.DataFrame({'Goals': pd.concat([home_goals, away_goals]), 'Type': ['Home']*len(home_goals) + ['Away']*len(away_goals)}) #make df for each goal to be home or away
    sns.boxplot(data=goals_data, x='Type', y='Goals', hue='Type') #plot goal distribution
    plt.title(f'Goals Distribution (Home vs Away) for {club.title()} in {season} Season')

    plt.subplot(3, 2, 4) #make fourth viz
    club_df['goal_difference'] = club_df.apply(lambda x: x['gh'] - x['ga'] if x['home_team'] == club else x['ga'] - x['gh'], axis=1) # make goal difference field
    sns.histplot(data=club_df, x='goal_difference', bins=8) #plot goal difference distribution
    plt.title(f"Goal Difference Distribution for {club.title()} in ({season}) Season")

    plt.subplot(3, 2, 5) #make fifth viz
    club_df['minute'] = pd.to_numeric(club_df['time'], errors='coerce') #convert time field to numeric for histplot
    sns.histplot(club_df['minute'].dropna(), bins=range(0, 100, 5), kde=True, color='purple') #plot goal time distribution
    plt.title(f"Goal Times for {club.title()} in ({season}) Season")

    plt.subplot(3, 2, 6) #make sixth viz
    club_df['won'] = club_df['winner'] == club #get club wins
    club_df = club_df.sort_values('date_dt') #sort date of results
    club_df['cumulative_wins'] = club_df['won'].cumsum() #get running total of wins throughout season
    sns.lineplot(data=club_df, x='date_dt', y='cumulative_wins', marker='o') #plot cumulative wins across dates
    plt.title(f"Cumulative Wins for {club.title()} in ({season}) Season")
    
season_options = sorted(gs['season'].dropna().astype(str).unique()) #make dropdown options for interact
club_options = sorted(set(gs['home_team'].dropna().unique()) | set(gs['away_team'].dropna().unique())) #make dropdown options for interact

@interact_manual(club=widgets.Dropdown(options=club_options, description='Club:'), season=widgets.Dropdown(options=season_options, description='Season:'))
def onclick(club, season):
    club = club.lower().strip() #match to df format
    
    #filter data
    season_df = gs[gs['season'] == season]
    club_df = season_df[(season_df['home_team'] == club) | (season_df['away_team'] == club)].copy()
    club_df['scoring_team'] = club_df['scoring_team'].str.strip().str.lower()

    if club_df.empty:
        print("No data available for this club and season.")
        return
    visualize(club_df, club, season)